In [1]:

import sys
import os

# Add project root to sys.path
sys.path.append(os.path.abspath(".."))

In [2]:
from utils.file_manager import FileManager
from data import DatasetPreprocessor, load_wic_dataset, preprocess_wic
from models import AutoTokenizer, AutoModel, load_model_and_tokenizer
from probing import ProbingClassifier

file_manager = FileManager()

/Users/juliahagen/anaconda3/envs/llm_rsa_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load model
model_name = "bert-base-uncased"
model, tokenizer = load_model_and_tokenizer(model_name)

# Create preprocessor for WiC
wic_preprocessor = DatasetPreprocessor(
    dataset_loader=load_wic_dataset,
    tokenizer=tokenizer,
    preprocess_fn=preprocess_wic
)

# Load and prepare the WiC dataset
texts, labels = wic_preprocessor.load_and_prepare(split="train")
texts, labels = texts[:100], labels[:100]


In [4]:
import random

#randomly choose mock layer
num_layers = model.config.num_hidden_layers
chosen_layer = random.randint(0, num_layers - 1)
print(f"Testing layer: {chosen_layer}")

Testing layer: 6


In [5]:

# Extract activations for the chosen layer
from models import get_activations  # Adjust import path as needed

layer_activations = get_activations(model, tokenizer, texts)[chosen_layer]

# Check activation shape
print(f"Layer {chosen_layer} Activations Shape: {layer_activations.shape}")

# Prepare data for probing classifier
probing = ProbingClassifier()
X, y = probing.prepare_data(layer_activations, labels)

# Train and evaluate
accuracy, f1 = probing.train(X, y)
print(f"Probing Classifier Results for Layer {chosen_layer}:")
print(f"Accuracy: {accuracy:.3f}, F1 Score: {f1:.3f}")

Layer 6 Activations Shape: torch.Size([100, 61, 768])
Probing Classifier Results for Layer 6:
Accuracy: 0.400, F1 Score: 0.400
